In [1]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
# import torch

# Read json files

In [2]:
alert_df=pd.read_json("../data/alerts_cleaned.json")

In [3]:
corazon_df= pd.read_json("../data/notification-labels-to-alert-surrogate-ids.json")

In [4]:
notification_df=pd.read_json("../data/notifications.json")

# Data Frame Cleaning

## Merging alert df with agency dummy variables

In [5]:
alert2= alert_df[['surrogate_id','agency']].set_index('surrogate_id')
alert2= pd.get_dummies(alert2)
grouped_alert= alert2.groupby(by='surrogate_id').sum()
alert_df.drop(['agency'], axis=1, inplace=True)
alert_df.drop_duplicates(subset='surrogate_id',inplace=True,ignore_index=True)
alert_df= pd.merge(alert_df,grouped_alert, on='surrogate_id')

## Merging notification df with corazon df

In [6]:
notification_df.drop_duplicates(inplace=True,ignore_index=True)
corazon_df.drop(['day'], axis=1, inplace=True)
corazon_df.drop_duplicates(inplace=True, ignore_index=True)
real_merge= notification_df.merge(corazon_df, how="left", left_on="join_key_value", right_on="notification_label_id")

## Merging all df together

In [7]:
real_merge.shape

(2274889, 8)

In [8]:
#Queried df for proper merging
label_merger= real_merge.query("join_field == 'label'")
alert_id_merger= real_merge.query("join_field == 'alertId'")

In [9]:
clean_df=label_merger.merge(alert_df, how="left",left_on="corazon_surrogate_id", right_on="surrogate_id")

In [10]:
clean_df_3=alert_id_merger.merge(alert_df, how="left",left_on="join_key_value", right_on="document_id")

In [11]:
clean_df_2=clean_df[clean_df['description'].isnull()]
clean_df.dropna(subset=['description'],inplace=True)
clean_df_2=clean_df_2[["event_date","event_timestamp", "event_name","user_id","join_field","join_key_value"]]
clean_df_4=clean_df_2.merge(alert_df, how="left",left_on="join_key_value", right_on="document_id")
clean_df_4.dropna(subset=['created_at'],inplace=True)

#Error! Notification open cant be related to any alerts. Ask

In [13]:
# Finding more problems

In [14]:
clean_df_3["join_key_value"].value_counts()

null                    484
-MddArdZ4i8kL5p7wG0H    466
-MdYiDEUKL6_4QGni-7o    421
-Me5bbxeM43sR8EbuLd2    386
-Mdt2BT-2y05YXyJctt2    371
                       ... 
-Mde9L0oTy1vODzHW294      1
-Mb2OcjgCicoGbENOVIQ      1
-McEmQIbv558V8RxXBIU      1
-Me1JV-RKQX-qj-_7QU8      1
-Ma9-5EDz-3vPh4VZOSb      1
Name: join_key_value, Length: 2289, dtype: int64

In [15]:
clean_df_3.query("event_name == 'notification_opened'")["description"].isnull().value_counts()
# Ask Ellen, 700 are gone

False    2826
True      673
Name: description, dtype: int64

In [16]:
#Another merge

In [17]:
clean_df_3["description"].isnull().value_counts()

False    32180
True      4308
Name: description, dtype: int64

In [18]:
clean_df_3.dropna(subset=['description'],inplace=True)

In [19]:
almost_finished_df= pd.concat([clean_df,clean_df_4], axis=0)
finished_df= pd.concat([almost_finished_df,clean_df_3], axis=0)

In [20]:
finished_df.head()

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,corazon_surrogate_id,notification_label_id,document_id,surrogate_id,...,agency_GtvOEQAFZ0GtU6u4AXwvPg,agency_HE59N3RXM0q5vKu4AXlQZg,agency_JUR9bFXmVkWDHqu4AXaY0g,agency_JfA8Bw8Zp024Kqu4AXiSpQ,agency_MgUq5b9mOEunx6u4AXt_BA,agency_NuuRQ2I1Q0a50Kv-AVKlLA,agency_V2AIQQKgmUO3VazvAOA-Cw,agency_jLjibFoim0iwWau4AWoEdQ,agency_pky7jovXYkaw-awAAMrQ3g,agency_zCy9zG00HEqGeKu4AWZYNQ
5,20210529,1622240527928000,notification_receive,ac6d3,label,o3RXFQPA6xGpKOFIXMTiIg,4Np4EQPA6xGpKOFIXMTiIg,o3RXFQPA6xGpKOFIXMTiIg,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,20210529,1622240959952000,notification_dismiss,ac6d3,label,o3RXFQPA6xGpKOFIXMTiIg,4Np4EQPA6xGpKOFIXMTiIg,o3RXFQPA6xGpKOFIXMTiIg,-Map3NyDNGaNwnfF6F0B,4Np4EQPA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,20210529,1622248553374000,notification_receive,ac6d3,label,M1Mb7hDA6xGpKOFIXMTiIg,kGA96xDA6xGpKOFIXMTiIg,M1Mb7hDA6xGpKOFIXMTiIg,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,20210529,1622248568285001,notification_dismiss,ac6d3,label,M1Mb7hDA6xGpKOFIXMTiIg,kGA96xDA6xGpKOFIXMTiIg,M1Mb7hDA6xGpKOFIXMTiIg,-MapQ4JNaTLEWbF2EG-b,kGA96xDA6xGpKOFIXMTiIg,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10,20210529,1622258011584001,notification_receive,ac6d3,label,E4TxyCvA6xGpKOFIXMTiIg,YKlNxSvA6xGpKOFIXMTiIg,E4TxyCvA6xGpKOFIXMTiIg,-Maq63vqDUxQRawQ_3Au,YKlNxSvA6xGpKOFIXMTiIg,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [21]:
finished_df.columns

Index(['event_date', 'event_timestamp', 'event_name', 'user_id', 'join_field',
       'join_key_value', 'corazon_surrogate_id', 'notification_label_id',
       'document_id', 'surrogate_id', 'created_at', 'published_at',
       'closed_at', 'cause', 'effect', 'description', 'is_global',
       'area_of_effect_coordinates_latitude',
       'area_of_effect_coordinates_longitude', 'agency_GewRJAw5tUmC4Ku4AX1-SQ',
       'agency_GtvOEQAFZ0GtU6u4AXwvPg', 'agency_HE59N3RXM0q5vKu4AXlQZg',
       'agency_JUR9bFXmVkWDHqu4AXaY0g', 'agency_JfA8Bw8Zp024Kqu4AXiSpQ',
       'agency_MgUq5b9mOEunx6u4AXt_BA', 'agency_NuuRQ2I1Q0a50Kv-AVKlLA',
       'agency_V2AIQQKgmUO3VazvAOA-Cw', 'agency_jLjibFoim0iwWau4AWoEdQ',
       'agency_pky7jovXYkaw-awAAMrQ3g', 'agency_zCy9zG00HEqGeKu4AWZYNQ'],
      dtype='object')

## Dropping irrelevant columns

In [22]:
export_df=finished_df.drop(['event_date','join_field','join_key_value','is_global','corazon_surrogate_id','notification_label_id'], axis=1)

## Fixing the event_name column:

In [23]:
export_df["event_name"].replace("notification_receive","notification_received", inplace=True)

In [24]:
export_df["event_name"].replace("notification_open","notification_opened", inplace=True)

## Exporting the df to a json file, so it is easier to deal without the cleaning part.

In [25]:
export_df.reset_index(inplace=True, drop=True)

In [26]:
export_df.to_json("../data/cleaned_data.json",orient='columns')

In [27]:
!pip3 install pyarrow
import pyarrow.feather as feather
feather.write_feather(export_df, "../data/cleaned_data.feather")

In [28]:
export_df = feather.read_feather("../data/cleaned_data.feather")